In [ ]:
import plotly.graph_objs as go
import plotly.express as px
from plotly import tools, subplots
import numpy as np
import pandas as pd
import ipywidgets as widgets
import math
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from ipywidgets import interact, interactive, fixed, interact_manual
init_notebook_mode(connected=True)
import scipy.io as sio
import random
random.seed(123)

## Set derivatives directory

The `derivativesDir` variable below should point to `..../ds-venus/derivatives/qMRFLow/`.

In [ ]:
derivativesDir = '/tmp/ds-toronto/derivatives/qMRFlow/'

## Define some variables and a helper function

Please execute the following cell.

In [ ]:
sessions = ['vendor750rev','vendorPRIrev','vendorSKYrev',
           'rth750rev','rthPRIrev','rthSKYrev']
sides = ["negative","negative","negative","positive","positive","positive"]
pointpos = [0,0,-0.14,0.14,0,0]
convert_tag = {'rth750':'G1<sub>neutral</sub>','rthPRI':'S1<sub>neutral</sub>','rthSKY':'S2<sub>neutral</sub>',
               'vendor750':'G1<sub>native</sub>','vendorPRI':'S1<sub>native</sub>','vendorSKY':'S2<sub>native</sub>'}
colors = {'rth750':'rgba(255,8,30,1)','rthPRI':'rgba(255,117,0,1)','rthSKY':'rgba(255,220,0,1)',
         'vendor750':'rgba(0,75,255,1)','vendorPRI':'rgba(24,231,234,1)','vendorSKY':'rgba(24,234,141,1)'}
colors_line = {'rth750':'rgba(255,8,30,0.35)','rthPRI':'rgba(255,117,0,0.35)','rthSKY':'rgba(255,220,0,0.35)',
         'vendor750':'rgba(0,75,255,0.35)','vendorPRI':'rgba(24,231,234,0.35)','vendorSKY':'rgba(24,234,141,0.35)'}
def plot_ridgelines(subject,sessions,metric,region,colors,sides,sampN,lowlim,uplim):
    df = pd.DataFrame()
    fig = go.Figure()
    for jj in range(0,len(sessions),1):
        avg_array  = []
        std_array  =  []
        cur_region = derivativesDir + '/sub-' + subject  + '/ses-' + sessions[jj] + '/stat/sub-' + subject + '_ses-' + sessions[jj] + '_desc-' + region + '_metrics.mat'
        cur_contents = sio.loadmat(cur_region)
        cur_contents = cur_contents[metric]
        cur_contents = np.delete(cur_contents, np.where(cur_contents < lowlim))
        cur_contents = np.delete(cur_contents, np.where(cur_contents > uplim))
        cur_tag = sessions[jj][0:-3]
        cur_contents = random.sample(list(cur_contents),sampN)
        fig.add_trace(go.Violin(x=cur_contents,
            name = convert_tag[cur_tag],
            marker=dict(
            opacity = 0.7,    
            outliercolor='rgba(255, 255, 255, 0.6)',
            line=dict(
                outliercolor='rgba(219, 64, 82, 0.6)',
                outlierwidth=2)),
            line_color= colors[cur_tag],
            side=sides[jj],
            pointpos = pointpos[jj]))

    fig.update_layout(height=1000, width=1200)
    fig.update_layout(legend=dict(x=1, y=1,tracegroupgap=300,font=dict(color="white")))
    axis_template = dict(linecolor = 'white', 
                         showticklabels = True,
                         tickfont=dict(color="white",size=12), 
                         gridcolor = 'rgba(255,255,255,0.2)')
    #fig.update_xaxes(axis_template,tickvals=np.round(np.linspace(lowlim,uplim,10),1))
    fig.update_yaxes(axis_template,showgrid=False)
    fig.update_traces(orientation='h')
    fig.update_layout(paper_bgcolor = '#1c2a3d',plot_bgcolor = '#1c2a3d')
    fig.update_traces(meanline=dict(visible=True,color="white",width=3),
                      points= 'outliers', # show all points
                      jitter=0.1,
                      width=3)  # add some jitter on points for better visibility
                      #scalemode='count')
    fig.update_xaxes(range=[lowlim,uplim])
    return fig

## Visualize interactive KDE ridgelines 

* In the manuscript, `invivo3` was selected for visualization. You can select `invivo1` and `invivo2` as well. 
* In the `WM`, sampN was `37000`
* In the `GM`, sampN was `20000`
* For metric `T1` `lowlim: 0` and `uplim: 2.5`  (WM)
* For metric `MTR` `lowlim: 35` and `uplim: 70` (WM)
* For metric `MTsat` `lowlim: 1` and `uplim: 8` (WM)

In [ ]:
plot_ridgelines('invivo3',sessions,'T1','WM',colors,sides,37000,0.5,2)
#plot_ridgelines('invivo3',sessions,'MTR','WM',colors,sides,37000,35,70)
#plot_ridgelines('invivo3',sessions,'MTsat','WM',colors,sides,37000,1,8)
#plot_ridgelines('invivo3',sessions,'T1','GM',colors,sides,20000,0.5,2) 

## Shift function explanation (Fig 6a)

Visualize two normal distributions and mark decile locations for each.

In [ ]:
from scipy.stats import norm
# generate random numbers from N(0,1)

fig = go.Figure()


data_normal = norm.rvs(size=10000,loc=0.5,scale=1)
data_normal2 = norm.rvs(size=10000,loc=0,scale=2)

# Figure for ideal case 
#data_normal = norm.rvs(size=10000,loc=0,scale=1)
#data_normal2 = norm.rvs(size=10000,loc=0,scale=1)

fig.add_trace(go.Violin(x=data_normal,
    opacity = 1,    
    line_color= "crimson",
    side="positive",
    meanline=dict(visible=True,color="crimson",width=4),
    width=2
))
fig.add_trace(go.Violin(x=data_normal2,
    marker=dict(
    opacity = 0.3,    
    outliercolor='rgba(255, 255, 255, 0.6)',
    line=dict(
        outliercolor='rgba(219, 64, 82, 0.6)',
        outlierwidth=2)),
    line_color= "limegreen",
    side="positive",
    meanline=dict(visible=True,color="limegreen",width=4),
    width=2                    
))


fig.update_traces(orientation='h')
fig.update_layout(paper_bgcolor = 'white',plot_bgcolor = 'white',height=800, width=1200)
fig.update_traces(points=False)

In [ ]:
qs = pd.qcut(data_normal2, 10, labels=False)
aa = []
for ii in range(10):
    aa.append(np.median(data_normal2[qs==ii]))
fig.add_vrect(
    x0=aa[9] + 0.8, x1=aa[9]+0.9,
    fillcolor="limegreen", opacity=0.7,
    layer="above", line_width=0,
)

for ii in range(0,9,1):
    fig.add_vrect(
    x0=aa[ii], x1=aa[ii] + 0.02,
    fillcolor="limegreen", opacity=0.3,
    layer="above", line_width=0)
fig.add_vrect(
    x0=aa[0], x1=aa[0] + 0.1,
    fillcolor="limegreen", opacity=0.7,
    layer="above", line_width=0,
)
qs = pd.qcut(data_normal, 10, labels=False)
aa = []
for ii in range(10):
    aa.append(np.median(data_normal[qs==ii]))  
fig.add_vrect(
    x0=aa[9], x1=aa[9]+0.1,
    fillcolor="crimson", opacity=0.7,
    layer="above", line_width=0,
)
fig.add_vrect(
    x0=aa[0], x1=aa[0]+0.1,
    fillcolor="crimson", opacity=0.7,
    layer="above", line_width=0,
)
for ii in range(0,9,1):
    fig.add_vrect(
    x0=aa[ii], x1=aa[ii] + 0.02,
    fillcolor="crimson", opacity=0.3,
    layer="above", line_width=0)
fig.show()

# SNR

In [ ]:
colors = {'rth750':'rgba(255,8,30,1)','rthPRI':'rgba(255,117,0,1)','rthSKY':'rgba(255,220,0,1)',
         'vendor750':'rgba(0,75,255,1)','vendorPRI':'rgba(24,231,234,1)','vendorSKY':'rgba(24,234,141,1)'}
df = pd.DataFrame()

In [ ]:
df['signal'] = [1649,240,264,793,2330,2379,767,110,117,362,1023,1016]
df['noise'] = [1.68,0.34,0.33,0.66,1.64,1.65,1.66,0.32,0.34,0.6,1.72,1.71]
df['session'] = ["rth750","rthPRI","rthSKY","vendor750","vendorPRI","vendorSKY"]*2
df['acq'] = ["T1w"]*6 + ["PDw"]*6 
df['SNR'] = 10*np.log10(df['signal']/df['noise'])

In [ ]:
len(df)

In [ ]:
fig = go.Figure()

aa = list(map(str,np.round(df['SNR'][0:6],2)))
bb = np.core.defchararray.add(['<br>']*6,df['acq'][0:6])
cc = np.core.defchararray.add(list(map(str,np.round(df['SNR'][0:6],2))), bb)
fig.add_trace(go.Bar(
        y=df['SNR'][0:6],
        x=df["session"][0:6],
        name="T1w",
        text=cc,
        textposition='auto',
        marker_color= [colors[x] for x in df['session']]))
aa = list(map(str,np.round(df['SNR'][6:12],2)))
bb = np.core.defchararray.add(['<br>']*6,df['acq'][6:12])
cc = np.core.defchararray.add(list(map(str,np.round(df['SNR'][6:12],2))), bb)
fig.add_trace(go.Bar(
        y=df['SNR'][6:12],
        x=df["session"][6:12],
        textposition='auto',
        text=cc,
        name="PDw",
        marker_color=[colors[x] for x in df['session']]))
fig.update_layout(barmode='group')
fig.update_layout(height=300, width=1000)

In [ ]:
plot(fig)

In [ ]:
aa = list(map(str,np.round(df['SNR'][0:6],2)))
bb = np.core.defchararray.add(['<br>']*6,df['acq'][0:6])
np.core.defchararray.add(list(map(str,np.round(df['SNR'][0:6],2))), bb)

In [ ]:
['<br>']*6